# Setup


In [1]:
import os
import cv2
import math
import pafy
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from moviepy.editor import *
from collections import deque
from enum import Enum
import csv
import matplotlib.pyplot as plt
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

seed_constant = 507
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

2023-12-03 16:48:56.877388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# With raw data

In [53]:
data_path = '../data/steps_in_3s/'



class cat:
    front = 0
    heel = 1
    noise = 2
cat_count = [0,0,0]
test_input, train_input = [], []
test_labels, train_labels = [], []
# test_input = np.array(test_input)
# train_input = np.array(train_input)
input_count = 0
max_input = -1


for file_name in os.listdir(data_path):
    f_path = os.path.join(data_path, file_name)
    if os.path.isfile(f_path):
        f = open(f_path,'r')
        cat_str = f_path.split("ST")[1][0]
        cat_type = cat.noise
        speed = f_path.split("P")[1][0]
        if(cat_str == 'f'):
            cat_type = cat.front
        elif(cat_str == 'h'):
            cat_type = cat.heel
        row_num = 1
        rows = f.readlines()
        if not (cat_type == cat.noise):
            for r in rows:
                #print(r)
                triplet = np.abs(np.array(r.split(',')).astype(np.float64))
                # print(np.size(triplet))
                # if(np.size(triplet) == 99):
                #     print("99 on row " + str(row_num) + " of " + f_path)
                #     print(triplet)
                row_num += 1
                input_count += 1
                cat_count[cat_type] += 1

                # check for new max
                temp_max = np.max(triplet)
                if(temp_max > max_input):
                    max_input = temp_max

                if speed == "4":
                    test_input.append(triplet)
                    # if(np.shape(test_input)[0] == 0):
                    #     test_input = triplet
                    # else:
                    #     test_input = np.vstack([test_input,triplet])
                    test_labels.append(cat_type)
                else:
                    train_input.append(triplet)
                    # if(np.shape(train_input)[0] == 0):
                    #     train_input = triplet
                    # else:
                    #     train_input = np.vstack([train_input,triplet])
                    train_labels.append(cat_type)
        f.close()

#normalize data
# train_input = train_input / max_input
# test_input = test_input / max_input

print("total input: ", input_count)
print("total test data: ", len(test_labels))
print("total training data: ", len(train_labels))
print("input by category (f,h,n): ", cat_count)


total input:  1769
total test data:  168
total training data:  1601
input by category (f,h,n):  [897, 872, 0]


In [57]:

from sklearn import svm
#clf = svm.NuSVC(gamma="auto")
#clf = svm.LinearSVC()
clf = svm.SVC()
clf.fit(train_input,np.array(train_labels))
test_pred = clf.predict(test_input)
print(np.sum(test_pred == test_labels)/len(test_labels))

0.8035714285714286


# With Filtered data and more features

In [51]:
data_path = '../data/steps_in_3s_clean/'



class cat:
    front = 0
    heel = 1
    noise = 2
cat_count = [0,0,0]
test_input, train_input = [], []
test_labels, train_labels = [], []
# test_input = np.array(test_input)
# train_input = np.array(train_input)
input_count = 0
max_input = -1


for file_name in os.listdir(data_path):
    f_path = os.path.join(data_path, file_name)
    if os.path.isfile(f_path):
        f = open(f_path,'r')
        cat_str = f_path.split("ST")[1][0]
        cat_type = cat.noise
        speed = f_path.split("P")[1][0]
        if(cat_str == 'f'):
            cat_type = cat.front
        elif(cat_str == 'h'):
            cat_type = cat.heel
        row_num = 1
        rows = f.readlines()
        if not (cat_type == cat.noise):
            for r in rows:
                #print(r)
                triplet = np.abs(np.array(r.split(',')).astype(np.float64))
                # print(np.size(triplet))
                # if(np.size(triplet) == 99):
                #     print("99 on row " + str(row_num) + " of " + f_path)
                #     print(triplet)
                row_num += 1
                input_count += 1
                cat_count[cat_type] += 1

                # check for new max
                temp_max = np.max(triplet)
                if(temp_max > max_input):
                    max_input = temp_max

                if speed == "4":
                    test_input.append(triplet)
                    # if(np.shape(test_input)[0] == 0):
                    #     test_input = triplet
                    # else:
                    #     test_input = np.vstack([test_input,triplet])
                    test_labels.append(cat_type)
                else:
                    train_input.append(triplet)
                    # if(np.shape(train_input)[0] == 0):
                    #     train_input = triplet
                    # else:
                    #     train_input = np.vstack([train_input,triplet])
                    train_labels.append(cat_type)
        f.close()

#normalize data
# train_input = train_input / max_input
# test_input = test_input / max_input

print("total input: ", input_count)
print("total test data: ", len(test_labels))
print("total training data: ", len(train_labels))
print("input by category (f,h,n): ", cat_count)

# print(np.shape(np.array(train_labels)))

# print(np.shape(train_input))

total input:  3198
total test data:  292
total training data:  2906
input by category (f,h,n):  [1612, 1586, 0]


# SVM


In [52]:
#print(train_input)
#define Nonlinear SVM
#clf = svm.NuSVC(gamma="auto")
#clf = svm.LinearSVC()
clf = svm.SVC()
clf.fit(train_input,np.array(train_labels))
test_pred = clf.predict(test_input)
print(np.sum(test_pred == test_labels)/len(test_labels))



0.8356164383561644
